In [ ]:
import nltk
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [ ]:
def preprocess_text(text):
    # Tokenization and converting to lowercase
    tokens = text.lower().split()
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing to the dataset
preprocessed_data = [preprocess_text(text) for text in newsgroups.data]

# Print a preprocessed document as an example
print(preprocessed_data[0])

sure bashers pens fans pretty confused lack kind posts recent pens massacre devils. actually, bit puzzled bit relieved. however, going put end non-pittsburghers' relief bit praise pens. man, killing devils worse thought. jagr showed much better regular season stats. also lot fo fun watch playoffs. bowman let jagr lot fun next couple games since pens going beat pulp jersey anyway. disappointed see islanders lose final regular season game. pens rule!!!


In [ ]:
# Create a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed data
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_data)

# Print the shape of the term-document matrix
print("Shape of the term-document matrix:", tfidf_matrix.shape)

Shape of the term-document matrix: (18846, 134409)


In [ ]:
# Define the number of topics (components) to retain
num_topics = 100

# Apply SVD to the term-document matrix
svd = TruncatedSVD(n_components=num_topics)
lsa_matrix = svd.fit_transform(tfidf_matrix)
# Print the shape of the LSA matrix
print("Shape of the LSA matrix:", lsa_matrix.shape)


Shape of the LSA matrix: (18846, 100)


In [ ]:
# Get singular vectors and terms
singular_vectors = svd.components_
feature_names = tfidf_vectorizer.get_feature_names_out()

# Find the indices of the top singular values
top_topics = [i for i in range(num_topics)]
topics_to_analyze = [0, 1, 2, 3, 4]
# Analyze the topics and their top terms
for topic_num in topics_to_analyze:
    # Get the top terms for this topic
    top_term_indices = singular_vectors[topic_num].argsort()[::-1][:10]
    top_terms = [feature_names[i] for i in top_term_indices]

    print(f"Topic {topic_num}:")
    print(", ".join(top_terms))
    print()


Topic 0:
would, one, like, people, know, it, get, think, could, time

Topic 1:
windows, thanks, drive, card, dos, please, file, anyone, mail, software

Topic 2:
god, jesus, windows, bible, thanks, christ, christian, faith, christians, believe

Topic 3:
drive, god, scsi, ide, card, hard, disk, controller, drives, game

Topic 4:
key, drive, government, chip, scsi, system, encryption, use, clipper, keys



In [ ]:
query = ["Hello world"]  # Place the query text inside a list

# Preprocess the query
query_tfidf = tfidf_vectorizer.transform(query)

# Project the query into LSI space using the singular vectors
query_lsi = svd.transform(query_tfidf)

# Compute cosine similarity between the query and all documents
cosine_similarities = cosine_similarity(query_lsi, lsa_matrix)  # Use lsa_matrix, which contains LSI-transformed documents

# Sort documents by similarity in descending order
most_similar_indices = cosine_similarities[0].argsort()[::-1]

# Print the most relevant documents
top_k = 10  # You can adjust the number of top documents you want to retrieve
for i, idx in enumerate(most_similar_indices[:top_k]):
    print(f"Top-{i + 1} Document:")
    print(newsgroups.data[idx])  # Print the content of the document
    print(f"Cosine Similarity: {cosine_similarities[0][idx]}")
    print()



Top-1 Document:
Hello,
Cosine Similarity: 0.6995243252324693

Top-2 Document:
Hello,
Cosine Similarity: 0.6995243252324693

Top-3 Document:
hello testing



Cosine Similarity: 0.668525470825247

Top-4 Document:

Take up residence in a fantasy world. 

Cosine Similarity: 0.6094752789555774

Top-5 Document:


Note that the two tables don't talk about the same population.  One is
Fortune 1000 companies favoring the platform as their primary
application platform, the other is sales (to everyone, not just
Fortune 1000).  Fortune 1000 companies don't do a lot of development
with the Mac as their top platform.  I would expect that that would
explain the discrepancy.

-- 
"Insisting on perfect safety is for people who don't have the balls to live
 in the real world."   -- Mary Shafer, NASA Ames Dryden
Cosine Similarity: 0.5392580255883939

Top-6 Document:


You missed something.  I think it takes off vertically and is intended
to land the same way.

-- 
"Insisting on perfect safety is for peop

In [ ]:
# Clustering using K-Means and Evaluation
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import silhouette_score
# Apply K-Means clustering to the LSI-transformed data
n_clusters = 20  # Adjust the number of clusters as needed
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
predicted_labels = kmeans.fit_predict(lsa_matrix)
silhouette_avg = silhouette_score(lsa_matrix, predicted_labels)

print(f"Silhouette Score:, {silhouette_avg:.4f}")
# Calculate Normalized Mutual Information (NMI) with the true labels (true_labels) and predicted cluster assignments
nmi = normalized_mutual_info_score(true_labels, predicted_labels)

print(f"Normalized Mutual Information (NMI):, {nmi:.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Silhouette Score:, 0.0492
Normalized Mutual Information (NMI):, 0.3072
